### Loading Data

In [231]:
import os
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold
import networkx as nx
from torch_geometric.utils import from_networkx

# Define folder path and TSV file containing labels
participants_xlsx_file = '.\\data\\participants.xsls'
participants_tsv_file = '.\\data\\participants.tsv'

# Load TSV file
df_labels = pd.read_csv(participants_tsv_file, sep='\t')

# Filter out rows where 'group' is 'n/a'
df_labels = df_labels[df_labels['group'].notna() & (df_labels['group'] != 'n/a')]

# Extract labels as a dictionary and convert to integers
labels_dict = dict(zip(df_labels['participant_id'], df_labels['group'].astype(int)))

# Function to load data
def load_data(node2vec_folder, graph_shapes_file, csv_prefix, csv_suffix, gml_prefix, gml_suffix):
    data_list = []  # List to store loaded data
    missing_files = []  # List to track missing files
    processed_subs = []  # List to track processed subjects

    for sub_id, label in labels_dict.items():
        # Correct the format of the subject ID to match the file format
        sub_id_corrected = sub_id.replace('sub-', 'Sub')
        csv_file = os.path.join(node2vec_folder, f"Node2Vec_{csv_prefix}_{sub_id_corrected}_{csv_suffix}.csv")
        gml_file = os.path.join(graph_shapes_file, f"GNN input_{gml_prefix}_{sub_id_corrected}_{gml_suffix}.gml")

        # Check if the file exists
        if os.path.exists(csv_file) and os.path.exists(gml_file):
            # Load node features from CSV
            node_features = pd.read_csv(csv_file, header=0, index_col=0).values
            num_nodes, num_features = node_features.shape

            # Load the graph structure from GML file
            nx_graph = nx.read_gml(gml_file)
            data = from_networkx(nx_graph)
            edge_index = data.edge_index

            # Create a complete graph for edge_index
            # edge_index = torch.combinations(torch.arange(num_nodes), 2).t()
            
            # Convert data to PyTorch tensors
            x = torch.tensor(node_features, dtype=torch.float)
            y = torch.tensor([label - 1], dtype=torch.long)  # Convert labels to 0 and 1
            
            # Create a Data object for PyTorch Geometric
            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
            processed_subs.append(sub_id)
        else:
            missing_files.append(sub_id)  # Add missing subject ID to the list

    return data_list





## Early stopping

In [232]:
# Define function for early stopping
def early_stopping(val_accuracies, patience):
    if len(val_accuracies) > patience:
        recent_accuracies = val_accuracies[-patience:]
        if max(recent_accuracies) <= val_accuracies[-patience-1]:
            return True
    return False

## Hyperparameters

In [233]:

# Hyperparameter search with Grid Search
param_grid = {
    'hidden_channels': [16, 32, 64],
    'dropout': [0.2, 0.4],
    'lr': [0.01, 0.0001],
    'weight_decay': [1e-2, 1e-4]
}

grid = ParameterGrid(param_grid)

## Evaluation

In [234]:
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, accuracy_score
from sklearn.metrics import confusion_matrix, average_precision_score, mean_squared_error, precision_recall_curve, auc as sklearn_auc


def compute_metrics(preds, labels):

    labels = labels.cpu().detach().numpy()
    preds = preds.cpu().detach().numpy()
    preds_binary = (preds > 0.5).astype(int)
    

    accuracy = accuracy_score(labels, preds_binary)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds_binary, average='binary', zero_division=0)
    auc = roc_auc_score(labels, preds)
    mse = mean_squared_error(labels, preds)
    conf_matrix = confusion_matrix(labels, preds_binary)
    average_precision = average_precision_score(labels, preds)
    

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "auc": auc,
        "accuracy": accuracy,
        "conf_matrix": conf_matrix,
        "ap": average_precision,
        "mse": mse,
    }


# Evaluation function
def evaluate(loader, model):
    model.eval()
    with torch.no_grad():
        for data in loader:
            out = model(data)
            res = compute_metrics(out, data.y)
            
    return res


In [235]:
import csv
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from openpyxl.styles import Font


# current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


def save_results(file_name, test_metrics, train_metrics, val_metrics, params):
    """
    Save cross-validation train, validation, and test metrics along with hyperparameters 
    into a single Excel file with separate sheets for train, validation, and test results.

    Parameters:
    - file_name: Name of the file (without extension).
    - train_metrics: List of train metrics for each fold.
    - val_metrics: List of validation metrics for each fold.
    - test_metrics: Dictionary of test metrics.
    - params: Dictionary of hyperparameters.
    """
    def reorder_columns(df):
        """Ensure the 'Fold' column is the first column."""
        columns = ["Fold"] + [col for col in df.columns if col != "Fold"]
        return df[columns]

    def create_df_with_mean(metrics_list, params):
        # Create a DataFrame for metrics
        df = pd.DataFrame(metrics_list)
        df["Fold"] = range(1, len(metrics_list) + 1)  # Add fold numbers
        for key, value in params.items():
            df[key] = value  # Add hyperparameters as columns
        
        # Calculate and append mean metrics
        mean_row = df.mean(numeric_only=True).to_dict()  # Calculate numeric means
        mean_row["Fold"] = "Mean"
        df = pd.concat([df, pd.DataFrame([mean_row])], ignore_index=True)

        # Reorder columns to make "Fold" the first column
        df = reorder_columns(df)
        return df

    # Create DataFrames for train, validation, and test metrics
    train_df = create_df_with_mean(train_metrics, params)
    val_df = create_df_with_mean(val_metrics, params)
    test_df = create_df_with_mean(test_metrics, params)  # Single test set treated as one fold

    # Save results into an Excel file with separate sheets
    current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    results_path = f"{file_name}_{current_time}.xlsx"
    with pd.ExcelWriter(results_path) as writer:
        train_df.to_excel(writer, index=False, sheet_name="Train Results")
        val_df.to_excel(writer, index=False, sheet_name="Validation Results")
        test_df.to_excel(writer, index=False, sheet_name="Test Results")

    # Apply bold formatting to the "Mean" row
    def bold_mean_row(sheet_name, df):
        wb = load_workbook(results_path)
        sheet = wb[sheet_name]
        mean_row_index = len(df) + 1  # Excel rows are 1-indexed
        for col in range(1, len(df.columns) + 1):  # Loop through all columns
            cell = sheet.cell(row=mean_row_index, column=col)
            cell.font = Font(bold=True)
        wb.save(results_path)

    # Apply bold formatting for each sheet
    bold_mean_row("Train Results", train_df)
    bold_mean_row("Validation Results", val_df)
    bold_mean_row("Test Results", test_df)

    print(f"Results saved to {results_path}")



def plot_average_auc(fileName, train_aucs_all, val_aucs_all):
    # Convert lists of lists to numpy arrays for easier manipulation
    train_aucs_all = np.array(train_aucs_all)
    val_aucs_all = np.array(val_aucs_all)
    
    # Compute mean and standard deviation across folds for each epoch
    mean_train_aucs = np.mean(train_aucs_all, axis=0)
    # std_train_aucs = np.std(train_aucs_all, axis=0)

    mean_val_aucs = np.mean(val_aucs_all, axis=0)
    # std_val_aucs = np.std(val_aucs_all, axis=0)

    epochs = range(1, len(mean_train_aucs) + 1)

    # Plot the average AUC curves with shaded areas for standard deviation
    plt.plot(epochs, mean_train_aucs, label='Average Train AUC', color='blue')
    # plt.fill_between(epochs, mean_train_aucs - std_train_aucs, mean_train_aucs + std_train_aucs, color='blue', alpha=0.2)

    plt.plot(epochs, mean_val_aucs, label='Average Validation AUC', color='orange')
    # plt.fill_between(epochs, mean_val_aucs - std_val_aucs, mean_val_aucs + std_val_aucs, color='orange', alpha=0.2)

    # Add labels, title, and legend
    plt.xlabel('Epochs')
    plt.ylabel('AUC')
    plt.title('Average AUC Curves Across Folds')
    plt.legend(loc='best')
    plt.grid(True)
    plt.savefig(f'./results/AverageAUC_{fileName}_{current_time}.png')
    plt.close()    


## Different GNN Models

In [236]:
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, GINConv
from torch_geometric.nn import global_mean_pool, global_add_pool, global_max_pool
from torch_geometric.nn import BatchNorm
from torch.nn import Sequential, Linear, ReLU

class GATModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=4, dropout=0.3):
        super(GATModel, self).__init__()
        
        # Define 3 GATConv layers with residual connections and batch normalization
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        self.bn1 = BatchNorm(hidden_channels * heads)
        self.res_proj1 = nn.Linear(in_channels, hidden_channels * heads)
        
        self.conv2 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, dropout=dropout)
        self.bn2 = BatchNorm(hidden_channels * heads)
        
        self.conv3 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=dropout)
        self.bn3 = BatchNorm(out_channels)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First layer with residual connection
        res1 = x
        res1 = self.res_proj1(res1)
        x = self.conv1(x, edge_index)
        x = self.bn1(x).relu()
        x = self.dropout(x) + res1

        # Second layer with residual connection
        res2 = x
        x = self.conv2(x, edge_index)
        x = self.bn2(x).relu()
        x = self.dropout(x) + res2

        x = self.conv3(x, edge_index)
        x = self.bn3(x)

        # Global mean pooling
        x = global_mean_pool(x, batch)
        x = torch.sigmoid(x)
        x = x.squeeze(-1)
        return x

class GCNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout):
        super(GCNModel, self).__init__()

        # First GCN layer with residual connection
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = BatchNorm(hidden_channels)
        self.res_proj1 = nn.Linear(in_channels, hidden_channels)
        
        # Second GCN layer
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = BatchNorm(hidden_channels)
        
        # Third GCN layer with residual connection
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.bn3 = BatchNorm(hidden_channels)
        self.res_proj3 = nn.Linear(hidden_channels, hidden_channels)

        # Final GCN layer to map to output channels
        self.conv_out = GCNConv(hidden_channels, out_channels)
        self.bn_out = BatchNorm(out_channels)

        # Fully connected layers after pooling
        self.fc1 = nn.Linear(out_channels, hidden_channels)
        self.fc2 = nn.Linear(hidden_channels, out_channels)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN layer with residual connection
        res1 = self.res_proj1(x)
        x = self.conv1(x, edge_index).relu()
        x = self.bn1(x).relu() + res1  # Residual connection
        x = self.dropout(x)

        # Second GCN layer
        x = self.conv2(x, edge_index).relu()
        x = self.bn2(x).relu()
        x = self.dropout(x)
        
        # Third GCN layer with residual connection
        res3 = self.res_proj3(x)
        x = self.conv3(x, edge_index).relu()
        x = self.bn3(x).relu() + res3  # Residual connection
        x = self.dropout(x)
        
        # Final GCN layer
        x = self.conv_out(x, edge_index).relu()
        x = self.bn_out(x).relu()
        x = self.dropout(x)

        # Global pooling
        x = global_mean_pool(x, batch)

        # Fully connected layers
        x = self.fc1(x).relu()
        x = self.dropout(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)

        return x.squeeze(-1)

class GraphSAGEModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout):
        super(GraphSAGEModel, self).__init__()
        
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.bn1 = BatchNorm(hidden_channels)
        
        self.conv2 = SAGEConv(hidden_channels, out_channels)
        self.bn2 = BatchNorm(out_channels)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First layer
        x = self.conv1(x, edge_index).relu()
        x = self.bn1(x)
        x = self.dropout(x)

        # Second layer
        x = self.conv2(x, edge_index).relu()
        x = self.bn2(x)
        x = self.dropout(x)

        # Global mean pooling
        x = global_mean_pool(x, batch)

        # Final output with sigmoid
        x = torch.sigmoid(x)
        return x.squeeze(-1)

class GINModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout):
        super(GINModel, self).__init__()
        
        # Define the MLP for GINConv layers
        def make_mlp(input_dim, hidden_dim, output_dim):
            return Sequential(
                Linear(input_dim, hidden_dim),
                ReLU(),
                Linear(hidden_dim, output_dim),
            )

        # Define 3 GINConv layers
        self.conv1 = GINConv(make_mlp(in_channels, hidden_channels, hidden_channels))
        self.bn1 = BatchNorm(hidden_channels)
        
        self.conv2 = GINConv(make_mlp(hidden_channels, hidden_channels, out_channels))
        self.bn2 = BatchNorm(out_channels)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First layer
        x = self.conv1(x, edge_index).relu()
        x = self.bn1(x)
        x = self.dropout(x)

        # Second layer
        x = self.conv2(x, edge_index).relu()
        x = self.bn2(x)
        x = self.dropout(x)

        # Global mean pooling
        x = global_mean_pool(x, batch)

        # Final output with sigmoid
        x = torch.sigmoid(x)
        return x.squeeze(-1)

## Rum Model

In [237]:
def run_model(data_list, model_name, fileName_base):

    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)

        
    train_data, test_data = train_test_split(data_list, test_size=0.15, random_state=42)
    test_loader = DataLoader(test_data, batch_size=len(test_data), shuffle=False)
    
    # Define k-fold cross-validation
    K = 5  # Number of folds
    kf = KFold(n_splits=K, shuffle=True, random_state=42)

    # Define early stopping parameters
    PATIENCE = 5

    # Store overall best results
    best_overall_train_auc = 0.0
    best_overall_val_auc = 0.0
    best_overall_params = None

    # Hyperparameter tuning with k-fold cross-validation
    for params in grid:
        print(f"\nTesting parameters: {params}")
        

        fileName = f"./results/{fileName_base}/lr_{params['lr']}_wd_{params['weight_decay']}_drout_{params['dropout']}_hidChannels{params['hidden_channels']}"

        # Extract the directory path
        parent_dir = os.path.dirname(fileName)
        if parent_dir and not os.path.exists(parent_dir):
            os.makedirs(parent_dir)

        # Store fold results
        fold_train_aucs = []
        fold_val_aucs = []

        fold_train_metrics = []
        fold_val_metrics = []
        fold_test_metrics = []


        # k-fold cross-validation
        for fold, (train_index, val_index) in enumerate(kf.split(train_data)):
            print(f"\nFold {fold + 1}/{K}")
            # Split data into training and test sets for the current fold
            fold_train_data = [data_list[i] for i in train_index]
            fold_val_data = [data_list[i] for i in val_index]

            # Create DataLoader for training and test sets
            train_loader = DataLoader(fold_train_data, batch_size=len(fold_train_data), shuffle=False)
            val_loader = DataLoader(fold_val_data, batch_size=len(fold_val_data), shuffle=False)
            
            # Initialize the model with current hyperparameters
            model = model_name(in_channels=data_list[0].num_features,
                            hidden_channels=params['hidden_channels'],
                            out_channels=1,
                            dropout=params['dropout'])

            # Define loss function and optimizer with L2 regularization
            criterion = nn.BCEWithLogitsLoss()
            optimizer = torch.optim.Adam(model.parameters(),
                                        lr=params['lr'],
                                        weight_decay=params['weight_decay'])

            # Train the GCN model for the current fold
            num_epochs = 100

            train_metrics = []
            val_metrics = []
            test_metrics = []
            
            train_aucs = []
            val_aucs = []

            for epoch in range(num_epochs):
                model.train()
                total_loss = 0
                for data in train_loader:
                    optimizer.zero_grad()
                    out = model(data)
                    loss = criterion(out, data.y.float())
                    loss.backward()
                    optimizer.step()
                    total_loss = loss.item()

                # Calculate training and validation metrics for each epoch
                train_evals = evaluate(train_loader, model)
                train_metrics.append(train_evals)
                train_aucs.append(train_evals['auc'])

                val_evals = evaluate(val_loader, model)
                val_metrics.append(val_evals)
                val_aucs.append(val_evals['auc'])
        
                if epoch % 10 == 0:
                    print(f"Epoch {epoch}/{num_epochs}, Train Loss: {total_loss/len(train_loader):.4f}, "
                        f"Train Acc: {train_evals['accuracy']:.4f}, Val Acc: {val_evals['accuracy']:.4f}")

                # Check for early stopping
                if early_stopping(val_aucs, PATIENCE):
                    print("Early stopping triggered...")
                    break
            
            fold_train_aucs.append(train_aucs)
            fold_val_aucs.append(val_aucs)

            mean_train_metrics = {key: np.mean([epoch[key] for epoch in train_metrics]) for key in train_metrics[0].keys()}
            fold_train_metrics.append(mean_train_metrics)

            mean_val_metrics = {key: np.mean([epoch[key] for epoch in val_metrics]) for key in val_metrics[0].keys()}
            fold_val_metrics.append(mean_val_metrics)

            test_metrics = evaluate(test_loader, model)
            fold_test_metrics.append(test_metrics)
             
        # save evaluation results for each fold for further checking
        save_results(fileName, fold_test_metrics, fold_train_metrics, fold_val_metrics, params)



        # Calculate average train, validation, and test auc across all folds
        avg_train_auc = np.mean([item['auc'] for item in fold_train_metrics])
        avg_val_auc = np.mean([item['auc'] for item in fold_val_metrics])
        avg_test_auc = np.mean([item['auc'] for item in fold_test_metrics])

        print(f"\nAverage Train AUC: {avg_train_auc:.4f}")
        print(f"Average Validation AUC: {avg_val_auc:.4f}")
        print(f"Average Test AUC: {avg_test_auc:.4f}")

        # Update best overall results
        if avg_val_auc > best_overall_val_auc:
            best_overall_val_auc = avg_val_auc
            best_overall_train_auc = avg_train_auc
            best_overall_params = params

        
    print(f"\nBest overall parameters: {best_overall_params}")
    print(f"Best overall train AUC: {best_overall_train_auc:.4f}")
    print(f"Best overall val AUC: {best_overall_val_auc:.4f}")


In [ ]:
model_dict = {
    "GATModel": GATModel,
    "GCNModel": GCNModel,
    "GraphSAGEModel": GraphSAGEModel,
    "GINModel": GINModel
}

#set these parameters first
embedding_sizes=[4, 16, 64]
node2vec_folder_base = ".\\data\\graph_embeddings_"
graph_shapes_file = ".\\data\\graph_shapes_Yeo_17_mask"
csv_prefix ="PCMCI"
csv_suffix ="Yeo-17"
gml_prefix ="PCMCI"
gml_suffix ="Yeo-17"


for model_name, model_class in model_dict.items():
    for embedding_size in embedding_sizes:
        # Adjust the folder path for the current embedding size
        node2vec_folder = f"{node2vec_folder_base}{embedding_size}"
        
        print(f"\nRunning {model_name} with embedding size {embedding_size}...")
        
        # Load the data
        data_list = load_data(node2vec_folder, graph_shapes_file, csv_prefix, csv_suffix, gml_prefix, gml_suffix)
        
        print(f"Number of loaded samples: {len(data_list)}")

        # If no data was loaded, raise an error
        if len(data_list) == 0:
            raise ValueError(f"No data samples were loaded for embedding size {embedding_size}. Check the folder path or data format.")
        

        fileName_base = f"model_{model_class.__name__}_embeddingSize_{embedding_size}"
        # Run the model
        run_model(data_list, model_class, fileName_base)